In [1]:
import pandas as pd
import torch
import re
import ast
import datasets
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

import warnings
warnings.filterwarnings("ignore")

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "do_sample": False,
}

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
def create_message(question, verbose_answer):
    content = f"""
    You are asked to extract the numerical answer to a question given a more verbose answer.

    Here's an example:
    - Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
    - Verbose answer: First, we need to find out how many clips Natalia sold in May. Since she sold half as many clips in May as she did in April, we can calculate this by dividing the number of clips sold in April by 2: 
    Clips sold in May = Clips sold in April / 2
    Clips sold in May = 48 / 2
    Clips sold in May = 24
    Now, we can find the total number of clips sold in April and May by adding the number of clips sold in each month:
    Total clips sold = Clips sold in April + Clips sold in May
    Total clips sold = 48 + 24
    Total clips sold = 72
    So, Natalia sold 72 clips altogether in April and May.
    - Assistant: 72

    Now do the same for this question: "{question}", where the verbose answer is: ["{verbose_answer}"]. Assistant:
    """
    
    messages = [
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": content}
    ]
    
    return messages

In [10]:
df = pd.read_csv('results/cot-zeroshot.csv')
df = df.drop(columns=['Unnamed: 0'])
df.head()

,query,correct,answer
0,Natalia sold clips to 48 of her friends in Apr...,72,"First, we need to find out how many clips Nat..."
1,Weng earns $12 an hour for babysitting. Yester...,10,Step 1: Convert the babysitting time from min...
2,Betty is saving money for a new wallet which c...,5,Step 1: Calculate the amount Betty has alread...
3,"Julie is reading a 120-page book. Yesterday, s...",42,Step 1: Determine the number of pages Julie r...
4,James writes a 3-page letter to 2 different fr...,624,Step 1: Determine the number of pages James w...


In [18]:
prompts = []
outs = []
for i in range(len(df)):
  prompts.append(create_message(df['query'][i], df['answer'][i]))
  output = pipe(prompts[i], **generation_args)
  outs.append(output[0]['generated_text'])

In [19]:
prompt = create_message(df['query'][0], df['answer'][0])
output = pipe(prompt, **generation_args)
print(output[0]['generated_text'])

 72
